# Natural Language Processing: Building a Spell Checker 
    
      
This task will involve the creation of a spellchecking system and an evaluation of its performance. The corpus, `holbrook.txt` is an useful one for this NLP task at hand.
The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    

*<font color="blue">Note:</font> We first preprocess the text and use functions provided by NLTK package to build optimum performing spell checker system. Then evaluate it using few standard techniques*

We break down this task in multiple smaller tasks as follows:

## Task 1: Import and prepare text data

We start by building a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. We consider an example (shown below), where there is only an error in the 10th word and so the list of indexes is [9].

<b><font color="blue">Prepare for Evaluation:</font></b>  
We split the data into a test set of 100 lines and a training set.

In [17]:
data = []

#Read from file holbrook.txt
with open('holbrook.txt', 'r') as holbrook_txt:
    holbrook = holbrook_txt.read().strip()
holbrook_lines = holbrook.split('\n')

def find_mistakes(sent):
    '''
    Function that searches for spelling mistakes (| characters)
    input: sent - a line from the text
    output: a dcitionary of format (original line, corrected line, index of misspelt words)
    '''
    dict_corrected = {}
    #Token sentence into words
    #wordTokenizer = RegexpTokenizer(r'\w+')
    line_words = sent.split(' ')
    dict_corrected['original'] = line_words
    dict_corrected['corrected'] = line_words
    dict_corrected['indexes'] = []
    #list comprehension that summarises misspellings eg. [(1, ('siter','sister')), (2, ('go','goes'))]
    discrepancies = [(i, w.split('|')) for i,w in enumerate(line_words) if '|' in w]
    if len(discrepancies) > 0:
        original = line_words.copy()
        corrected = line_words.copy()
        #Loop through all misspellings in a single sentence
        for word_index, word in discrepancies:
            original[word_index] = word[0] #misspelt original word eg. siter
            corrected[word_index] = word[1] #corrected word eg. sister
            dict_corrected['indexes'].append(word_index)
        dict_corrected['original'] = original
        dict_corrected['corrected'] = corrected
    return(dict_corrected)

#execute the above defined function for all lines in the holbrook text
data = list(map(find_mistakes, holbrook_lines))
#Split train/ test/ correct/ incorrect sentences
test_correct = [sentence['corrected'] for sentence in data[:100]]
train_correct = [sentence['corrected'] for sentence in data[100:]]
test_incorrect = [sentence['original'] for sentence in data[:100]]
train_incorrect = [sentence['original'] for sentence in data[100:]]

#Test this unit of code
assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [18]:
test = data[:100]
train = data[100:]

## **Task 2**: Word Frequency Calculation 
Next, we calculate the frequency (number of occurrences), *ignoring case*, of all words and bigrams (sequences of two words) from the corrected *training* sentences:

In [19]:
from nltk.util import ngrams
from nltk.probability import FreqDist
import nltk

def unigram(word):
    '''Function that returns unigram frequency of the word in the corrected words training corpus'''
    fdist_unigrams = FreqDist([word.lower() for line in train_correct for word in line])
    return fdist_unigrams[word.lower()]
    

def bigram(words):
    '''Function that returns bigram frequency of the word in the corrected words training corpus'''
    fdist_bigrams = FreqDist(ngrams([word.lower() for line in train_correct for word in line],2))
    return fdist_bigrams[tuple(nltk.word_tokenize(words))]

# Test your code with the following
assert(unigram("me")==87)
assert(bigram("my mother")==17)
assert(bigram("you did")==1)

## **Task 3**: Compute  edit distance
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [20]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Using the above edit distance, we write a function that calculates all words with *minimal* edit distance to the misspelled word. The sub tasks involved are:
1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value


In [21]:

from nltk.metrics.distance import edit_distance

def get_candidates(token):
    '''
    function that calculates all words with minimal edit distance to the misspelled word
    input: token - misspelt word
    output: list of tokens with minimum edit distance
    '''
    
    #A set of unique tokens in the training data
    unique_tokens = set(tok.lower() for line in train_correct for tok in line)
    #Calculate edit distances between query token and all other tokens
    token_distances = [(edit_distance(token, w), w) for w in unique_tokens]
    #Find the minimum edit distance from the misspelt word
    min_distance = min(token_distances)[0]
    #return all unique tokens with minimum edit distances
    return [w for (_, w) in token_distances if _ == min_distance]
        
# Test your code as follows
assert(get_candidates("minde") == ['mind', 'mine'])

## Task 4: Rudimentary Spell Checker (Baseline version)

We build a rudimentary spell checker that takes a (misspelled) sentence and returns the corrected version of that sentence. The system scans the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *bigram probability*. That is the candidate should be selected using the previous and following word in a bigram language model. Thus, if the $i$th word in a sentence is misspelled we use the following to rank candidates:

$$p(w_{i+1}|w_i) p(w_i|w_{i-1})$$

For the first and last word of the sentence we use only the conditional probabilities that exist.
<font color="blue">Note:</font> We just define a baseline version of spell checker which we will enhance with additional natural language processing features and then compare the results.

In [22]:
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist
import nltk
import string
#Find all correct words from training samples
all_correct_words = [word for line in train_correct for word in line]
#Generate a set of all unique and correct words
all_unique_correct_words = [w for w in set(all_correct_words)]
#Create bigrams from the corpus words
holbrook_bigrams = nltk.bigrams(all_correct_words)
#Generate conditional frequency distribution of all bigrams over holbrook text
fd_bigrams = nltk.FreqDist(holbrook_bigrams)
#Generate conditional probability distribution from frequency distribution
cpd_bigrams = ConditionalProbDist(fd_bigrams, MLEProbDist)

In [23]:

def correct(incorrect_sentence):
    '''
    function that attempts to rectify misspelt words in the sentence
    input: a list of words in the incorrect sentence eg. ["this","whitr","cat"]
    output: a list of words (rectified) in the sentence eg. ["this","white","cat"]
    '''
    #Create a copy of list so that original is not modified
    sentence = incorrect_sentence.copy()
    
    #Word is classified as misspelt if it is not found in training corpus
    misspelt_words = [m for m in sentence if m not in all_unique_correct_words]
    
    #Iterate through each misspelt word and try to correct it
    for mw in misspelt_words:
        #Find words with minimum edit_distance to misspelt words
        #Calculate edit distances between misspelt word and all other words in words corpus
        nearest_words = get_candidates(mw)
        #Calculate bigram probabilities with all combinations from nearest dictionary words
        if sentence.index(mw) == 0:
            #if first word in sentence is misspelt, consider combination with only next word
            if len(sentence) > 1:
                probs = [(cpd_bigrams[w].prob(sentence[1]), w) for w in nearest_words]
        elif sentence.index(mw) == (len(sentence) - 1):
            #if last word in sentence is misspelt, consider combination with only previous word
            probs = [(cpd_bigrams[w].prob(sentence[sentence.index(mw) - 1]), w) for w in nearest_words]
        else:
            #if misspelt word is neither at start nor at end of sentence
            before = sentence[sentence.index(mw) - 1] #previous word
            after = sentence[sentence.index(mw) + 1] #next word
            probs = [(cpd_bigrams[before].prob(w) * cpd_bigrams[w].prob(after), w) \
                     for w in nearest_words]
        if len(sentence) > 1:
            #Get the most likely correct word from the key-value pair (probability, candidate_word)
            most_likely_correction = max(probs)[1]
            #Replace the misspelt word with the most likely candidate
            sentence[sentence.index(mw)] = most_likely_correction
    return sentence

#Test this unit
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5**: Evaluate the baseline spell-checker 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [14]:
def accuracy(test):
    '''
    Compute baseline accuracy of the spell checker
    input: a list of dictionary that has both, tokens of correct as well as incorrect words
    output: accuracy, a decimal value
    '''
    count_total = 0
    count_correct = 0
    
    for line in test:
        rectifiedSentence = correct(line['original'])
        for index, word in enumerate(line['corrected']):
            count_total += 1
            #iterate over all words to find how many words from the 
            #system's output match the corrected sentence
            if rectifiedSentence[index] == word:
                count_correct += 1
    accuracy = round((count_correct * 100/ count_total),3)    
    return accuracy

print('Baseline accuracy of the spellchecker is: {} %'.format(accuracy(test)))

Baseline accuracy of the spellchecker is: 82.197 %


## **Task 6: Enhancing the spell-checker with rich NLP features**

We now propose modification to the baseline algorithm (developed in Task 3 and 4) that would improve its accuracy.


### Proposed improvements to the algorithm:

### 6.1. Handle issues with proper nouns:
 - <b><font color="blue">Issue:</font></b> It was observed that system attempted to correct all proper nouns since they could not be found in dictionary and were considered as misspelt words. Algorithm failed miserably as it tried to rectify rarely occuring proper nouns (even those already correct) and returned irrelevant results, bringing down the overall accuracy.
 - <b><font color="green">Solution:</font></b> Misspellings in proper nouns can be difficult to correct by finding similar candidates as they tend to be rarely used in the training corpus. So, unless the proper nouns occur at least once in the training corpus, it would be impossible to correct them. Hence, all words were <b>*pos-tagged*</b> before searching in dictionary, and words excluding NNP and NNPS (proper nouns singular and plural) were seeked in the dictionary. In this way, algorithm was not allowed to compute suitable replacement candidates for these words and names those were already correct, were saved from being misspelt. Example follows:

<b><font color=blue>Baseline Algorithm:</font></b>

In [47]:
sentence = "I live at Boar Parva near Smallerden ."
' '.join(correct(sentence.split(' ')))

'I live at your area near taller .'

<b><font color="green">Improved Algorithm: </font></b>

In [48]:
sentence = "I live at Boar Parva near Smallerden ."
' '.join(correct_improved(sentence.split(' ')))

'I live at Boar Parva near Smallerden .'

### 6.2.  Handle contractions like (I'll, we'll, she'll, etc.)
- <b><font color="blue">Issue:</font></b> Some of the anomalies include word contractions with "will" and "shall", which have been misspelt without the necessary apostrophe (') punctuation mark. System fails to identify such words (*Ill* for "I will" or *well* for "we shall") and tries to correct them without any success.     
- <b><font color="green">Solution:</font></b> In English language, such contractions (specially with *double ll*) follow a grammar rule.  <br/> <b>Rule: </b>Generally, "<font color="red">ll</font>" contraction is attached to a <font color="blue">pronoun</font> and is followed by a <font color="green">verb</font>. eg. <font color="blue">We</font><font color="red">'ll</font> <font color="green">talk</font>, <font color="blue">I</font><font color="red">'ll</font> <font color="green">watch</font> TV, <font color="blue">she</font><font color="red">'ll</font> <font color="green">pay</font> the bill, etc.)<br/>
So, I tokenize the sentence and check the sentence for above rule in order to differentiate between actual contractions and words that end in double "ll" like "well", "ball", etc. If the sentence is seen to observe the rule then, such occurences are replaced with apostrophe "ll" ('ll), which are recognized as valid spellings by the system

<b><font color=blue>Baseline Algorithm:</font></b>

In [136]:
sentence = "MARY Oh Ill have ice-cream and what will you have"
' '.join(correct(sentence.split(' ')))

'MARY Oh ill have ice-cream and what will you have'

<b><font color="green">Improved Algorithm: </font></b>

In [131]:
sentence = "MARY Oh Ill have ice-cream and what will you have"
' '.join(correct_improved(sentence.split(' ')))

"MARY Oh I'll have ice-cream and what will you have"

In [24]:
from nltk.tokenize import word_tokenize

def handle_contractions(flawed_sentence):
    '''Function to detect valid contractions of double ll and replace them with apostrophe ll'''
    sentence = flawed_sentence.copy()
    #find words in sentence that end in "ll"
    mods = [(w, sentence.index(w)) for w in sentence if w.endswith('ll')]

    #Check grammar rule for each instance of above found word
    for mod, index_md in mods:
        #Extract word part preceding double ll
        removed_ll = mod[:-2]
        new_sentence = sentence.copy()
        #create a new sentence without ll. eg. [I'll, eat] becomes [I, eat]
        new_sentence[index_md] = removed_ll
        #pos tag the modified sentence
        pos_sent = nltk.pos_tag(new_sentence)
        if index_md > 1:
            previous_tag = pos_sent[index_md][1]
            if index_md != (len(new_sentence) - 1):
                next_tag = pos_sent[index_md + 1][1]
                #Check the grammar rule that states ll is preceded by pronoun and followed by a verb
                if next_tag.startswith('VB') and previous_tag.startswith('PRP'):
                    sentence[index_md] = "'".join([removed_ll,"ll"]) #add apostrophe if rule is obeyed
        else:
            next_tag = pos_sent[index_md + 1][1]
            if next_tag.startswith('VB'):
                sentence[index_md] = "'".join([removed_ll,"ll"])
    return sentence

assert(handle_contractions(word_tokenize("MARY Oh Ill have ice-cream and what will you have")) == ['MARY','Oh',"I'll",
 'have','ice-cream','and', 'what','will', 'you', 'have'])

### 6.3. Use external corpus as dictionary

- <b><font color="blue">Issue:</font></b> Some of the valid English words in unseen (heldout) test data that do not occur in training set (that we use as dictionary) are flagged as misspelt words. Then an attempt is made to correct that "misspelt" word which when fails, affects the accuracy of the spell checker   
- <b><font color="green">Solution:</font></b> Use of large corpus such as "words" can help detect if word is truly a valid English word or not and thus forbidding an attempt to correct it. Dictionary is enriched with English language words from larger corpora like <font color="brown">nltk.corpus.words</font> and <font color="brown">nltk.corpus.cmudict</font>

<b><font color=blue>Baseline Algorithm:</font></b>

In [154]:
sentence = "you will have to feed them on a bottle"
' '.join(correct(sentence.split(' ')))

'you will have to feed them on a little'

<b><font color="green">Improved Algorithm: </font></b>

In [155]:
sentence = "you will have to feed them on a bottle"
' '.join(correct_improved(sentence.split(' ')))

'you will have to feed them on a bottle'

### 6.4. Train with wider range of collocations using external corpus

- <b><font color="blue">Issue:</font></b> Some words that commonly co-occur may be missing from the training set, but are encountered in the test set. At such times, the system fails to utilize this sense of co-occurence and returns less probable candidates.
- <b><font color="green">Solution:</font></b> It would be better if the model is initially exposed to wider range of collocations in English language. Thus, model is trained with bigrams from popular and relatively larger <font color="brown">"brown"</font> corpus in addition to bigrams from Holbrook training set.

### 6.5. Good-Turing and Kneser–Ney smoothing

- <b><font color="blue">Issue:</font></b> Owing to *data sparsity* in training set, probabilities associated with some of the unigrams or bigrams turn up as 0. As calculation of conditional probability involves multiplication, the presense of even a single zero prior results in no likelihood of the candidate word. 
- <b><font color="green">Solution:</font></b> Smoothing is required to counter the problem of data sparsity and subsequent zero probabilities. Couple of smoothing techinques have been used for improving the spell-correction algorithm: 

<strong>1. Good Turing Estimation:</strong> It is used to reallocate the probability mass of n-grams that occur r + 1 times in the training data to the n-grams that occur r times. i.e. Some of the probability mass of n-grams that occur at least once is shifted to those that never occured. Thus, none of the n-grams are assigned 0 probabilities. NLTK supports good turing estimation with <font color="purple">"SimpleGoodTuringProbDist"</font> module.      

- NLTK  method signature:  
*<font color="purple">
nltk.SimpleGoodTuringProbDist(freqdist, bins=None)   
)   *  </font>    
<font color="red">:param freqdist:</font> The frequency counts upon which to base the
    estimation.       
<font color="red">:param bins:</font> The number of possible event types.     


In [ ]:
from nltk.util import ngrams

#Find unigram probability distribution with Simple Good Turing estimation
#Use all correct words from Holbrook as well sa brown corpus
fd_unigrams = nltk.FreqDist(all_correct_words + nltk.corpus.brown.words())
pd_unigrams = nltk.SimpleGoodTuringProbDist(fd_unigrams)

#Find bigram probability distribution with Simple Good Turing estimation
fd_bigrams = nltk.FreqDist(nltk.bigrams(all_correct_words))
#add bigram frequencies from holbrook and brown corpuses
fd_bigrams = fd_bigrams + nltk.FreqDist(ngrams(nltk.corpus.brown.words(),2))
pd_bigrams_smoothed = nltk.SimpleGoodTuringProbDist(fd_bigrams)

<strong>2. Kneser–Ney smoothing: </strong> This model uses the concept of absolute-discounting interpolation which incorporates information from higher and lower order language models. The addition of the term for lower order n-grams adds more weight to the overall probability when the count for the higher order n-grams is zero. Similarly, the weight of the lower order model decreases when the count of the n-gram is non zero.   
   
 - NLTK method signature:   
*<font color="purple">
nltk.KneserNeyProbDist(freqdist, bins=None, discount=0.75)  
)   *  </font>    
<font color="red">:param freqdist:</font> The trigram frequency distribution upon which to base
    the estimation   
<font color="red">:param bins:</font> The number of possible event types.   
<font color="red">:param discount:</font> The discount applied when retrieving counts of
    trigrams

In [ ]:
#Find trigram probability distribution with Kneser-Ney smoothing
fd_brown_trigrams = nltk.FreqDist(ngrams(nltk.corpus.brown.words(),3))
holbrook_trigrams = ngrams(all_correct_words,3)
fd_holbrook_trigrams = nltk.FreqDist(holbrook_trigrams)
#Combine trigrams from holbrook as well as brown corpus
cfd_trigrams = fd_brown_trigrams + fd_holbrook_trigrams
cpd_trigrams = nltk.KneserNeyProbDist(cfd_trigrams)


### 6.6. Linear interpolation of trigrams, bigrams and unigrams

- <b><font color="blue">Issue:</font></b> Certain trigrams are more popular than others. for eg. In sentence, 'I like watching TV', the trigram (to, watch, TV) has more probability of occurence than a trigram in an illogical but syntactically correct sentence 'I like washing TV'. However, this notion is not captured currently by only using bigram probabilities individually.

- <b><font color="green">Solution:</font></b>Trigram, bigram and unigram probabilities can be interpolated probability as:
\begin{equation*}
P(w|u,v) = \lambda_1 * P_M(w|u,v) + \lambda_2 * P_M (w|v) + \lambda_3 * P_M(w)
\end{equation*}   
Thus importance is given to probability distribution of the trigram, bigram as well as unigrams that forms the prior probabilities. So, with respect to example above, 


\begin{equation*}
0.7 * P(TV | like, washing) + 0.2 * P(TV|washing) + 0.1 * P(TV) < 0.7 * P(TV | like, watching) + 0.2 * P(TV|watching) + 0.1 * P(TV)   
\end{equation*}


In [212]:
#Create a dictionary of English words by taking words occurring in both 'words' as well as CMU dictionary corpus
wordDictionary = set(list(set([w[0] for w in nltk.corpus.cmudict.entries()]) & 
                             set([w.lower() for w in nltk.corpus.words.words()])))
dictionaryWords = [w for w in wordDictionary]
#Add words from brown corpus to dictionary
dictionaryWords += [w.lower() for w in set(nltk.corpus.brown.words())]

holbrookCorrectWords = [w for w in all_correct_words if w not in string.punctuation]
#Enrich the dictionary with words from Holbrook corpus and keep only unique ones
dictionaryAndHolbrookwords = set(dictionaryWords + holbrookCorrectWords)
def correct_improved(incorrect_sentence):
    '''Improved algorithm for spelling correction
    input: a list of word tokens from a sentence
    output: a list of 'corrected' word tokens of a sentence
    '''
    #create a local copy of original sentence to avoid changing it
    sentence = incorrect_sentence.copy()
    #Handle 'll contractions before further processing
    sentence = handle_contractions(sentence)
    
    #Find misspelt words. ie word that do not occur in enriched dictionary or are not digits/ punctuations
    misspelt_words = [m[0] for m in nltk.pos_tag(sentence) if m[1] not in ('NNP', 'NNPS') \
    and m[0].lower() not in dictionaryAndHolbrookwords and \
    m[0] not in string.punctuation and not m[0].isdigit()]
    #create a sentence object after stripping original punctuation marks
    temp_sentence = [w for w in sentence if w not in string.punctuation]
    
    #Iterate and try to rectify errors in each misspelt word
    for mw in misspelt_words:
        #Find edit distance from all correct words in holbrook training set
        token_distances = [(edit_distance(mw, w), w) for w in holbrookCorrectWords]
        #Find the minimum edit distance from the misspelt word
        min_distance = min(token_distances)[0]
        #Get all unique words that are at minimum distance from misspelt word
        nearest_words = set(w for (_, w) in token_distances if _ == min_distance)
        #Calculate the probabilities with all combinations from nearest dictionary words
        if temp_sentence.index(mw) == 0:
            #if the only word in sentence is misspelt, consider only it for probability
            if len(sentence) == 1:
                probs = [(pd_unigrams.prob(w), w) for w in nearest_words]
            if len(sentence) == 2:
            #if the sentence contains only 2 words and first word is misspelt
                probs = [(0.9 * pd_bigrams_smoothed.prob((w, sentence[0]))
                             + 0.1 * pd_unigrams.prob(w), w) for w in nearest_words]
            if len(sentence) > 2:
            #if the sentence contains more than 2 words and first word is misspelt
            #Calculate interpolated probabilities
                probs = [(0.7 * cpd_trigrams.prob((w, sentence[0], sentence[1]))
                          + 0.2 * pd_bigrams_smoothed.prob((w, sentence[0]))
                             + 0.1 * pd_unigrams.prob(w), w) for w in nearest_words]
        elif temp_sentence.index(mw) == (len(temp_sentence) - 1):
            #if last word in sentence is misspelt, consider combination with only previous 2 words
            probs = [(0.7 * cpd_trigrams.prob((sentence[-3], sentence[-2], w))
                      + 0.2 * pd_bigrams_smoothed.prob((sentence[-2], w))
                             + 0.1 * pd_unigrams.prob(w), w) for w in nearest_words]
        else:
        #if misspelt word is neither at start nor at end of sentence            
            msindex = temp_sentence.index(mw)
        
            if msindex == (len(temp_sentence) - 2):
            #if penultimate word in sentence is misspelt
                         #P(w+1|w) + P(w)
                probs = [((0.9 * pd_bigrams_smoothed.prob((w, temp_sentence[msindex + 1]))
                             + 0.1 * pd_unigrams.prob(w)) * \
                          #P(w-1|w,w+1) + [P(w|w-1)*P(w+1|w)] + P(w)
                          (0.8 * cpd_trigrams.prob((temp_sentence[msindex - 1], w, temp_sentence[msindex + 1]))
                          + 0.2 * (pd_bigrams_smoothed.prob((temp_sentence[msindex - 1], w)) 
                                   * pd_bigrams_smoothed.prob((w, temp_sentence[msindex + 1])))
                             + 0.1 * pd_unigrams.prob(w)) * \
                          #P(w|w-2,w-1) + P(w|w-1) + P(w) 
                          (0.7 * cpd_trigrams.prob((temp_sentence[msindex - 2], temp_sentence[msindex - 1], w))
                          + 0.2 * pd_bigrams_smoothed.prob((temp_sentence[msindex - 1], w))
                             + 0.1 * pd_unigrams.prob(w)), w) \
                         for w in nearest_words]
            else:
                #P(w+2|w,w+1) + [P(w+1|w)] + P(w) Misspelt word at the beginning of the trigram
                probs = [((0.7 * cpd_trigrams.prob((w, temp_sentence[msindex + 1], temp_sentence[msindex + 2]))
                          + 0.2 * pd_bigrams_smoothed.prob((w, temp_sentence[msindex + 1]))
                             + 0.1 * pd_unigrams.prob(w)) * \
                #misspelt word in middle of trigram
                (0.7 * cpd_trigrams.prob((temp_sentence[msindex - 1], w, temp_sentence[msindex + 1]))
                + 0.2 * (pd_bigrams_smoothed.prob((temp_sentence[msindex - 1], w)) 
                                   * pd_bigrams_smoothed.prob((w, temp_sentence[msindex + 1])))
                             + 0.1 * pd_unigrams.prob(w)) * \
                #misspelt word at the end of the trigram
                (0.7 * cpd_trigrams.prob((temp_sentence[msindex - 2], temp_sentence[msindex - 1], w))
                + 0.2 * pd_bigrams_smoothed.prob((temp_sentence[msindex - 1], w))
                             + 0.1 * pd_unigrams.prob(w)), w) \
                for w in nearest_words]
        if len(sentence) > 1:
            #The correct word is taken to be the one with highest probability mass
            most_likely_correction = max(probs)[1]
            #Replace the word in original sentence
            sentence[sentence.index(mw)] = most_likely_correction
    return sentence

## **Task 7: Evaluate the enchanced spell checker**

We repeat the evaluation (as in Task 5) of our new spell checking algorithm and show that it **outperforms** the baseline algorithm from Task 3 and 4

In [230]:
def accuracy_improved(test):
    '''Evaluate accuracy of the improved algorithm'''
    count_total = 0
    count_correct = 0
    lineNo = 0
    listCorrection = []
    for line in test:
        dictRectification = {}
        lineNo += 1
        rectifiedSentence = correct_improved(line['original'])
        dictRectification['ActuallyCorrect'] = line['corrected']
        dictRectification['Original'] = line['original']
        dictRectification['SystemRectified'] = rectifiedSentence
        #Compare each word in already correct sentence to that of system rectified sentence
        for index, word in enumerate(line['corrected']):
            count_total += 1
            if rectifiedSentence[index] == word:
                count_correct += 1
        listCorrection.append(dictRectification)
    accuracy = round((count_correct * 100/ count_total),3)    
    return accuracy

print(f'Accuracy of the improved algorithm is: {accuracy_improved(test)}%')

Accuracy of the improved algorithm is: 91.231%


### Conclusion:    
The improved algorithm is around 91.23 % accurate in spelling correction task, which is about <font color="green">11% better</font> than original algorithm [82.197%] (in task 3 & 4). Morever, it uses smoothing and interpolation techniques that have lower *perplexity* than the original algorithm

### References:    

1. Prof.Dr. John McCrae & Prof.Dr. Paul Buitelaar, Class notes in module CT5101 Natural Language Processing, National University of Ireland, Galway.
2. Stanley F. Chen and Joshua Goodman. 1999. An empirical study of smoothing techniques for language modeling. Comput. Speech Lang. 13, 4 (October 1999), 359-394. DOI=http://dx.doi.org/10.1006/csla.1999.0128
3. A. Gale, William & Sampson, Geoffrey. (1995). Good-Turing Frequency Estimation Without Tears. Journal of Quantitative Linguistics. 2. 217-237. 10.1080/09296179508590051. 
4. [NLTK Documentation](https://www.nltk.org/)
5. [https://stackoverflow.com/](https://stackoverflow.com/)